# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
weather_df = pd.read_csv("cities.csv")
weather_data_df = weather_df.dropna(how="any")
weather_data_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,khatanga,46.68,113.28,31.01,80.0,0.0,7.61,MN,1.602723e+09
1,1,kidal,71.97,102.50,18.07,97.0,100.0,11.50,RU,1.602724e+09
2,2,mulungu,18.44,1.41,86.34,14.0,0.0,9.28,ML,1.602724e+09
3,3,poum,-7.02,-35.46,77.00,83.0,40.0,6.93,BR,1.602724e+09
4,4,atambua,-20.23,164.02,75.56,81.0,100.0,5.79,NC,1.602724e+09
5,5,ushuaia,-9.11,124.89,87.19,39.0,53.0,2.04,ID,1.602724e+09
6,6,denpasar,-54.80,-68.30,48.20,49.0,75.0,13.87,AR,1.602724e+09
7,7,hermanus,-8.65,115.22,84.20,74.0,75.0,14.99,ID,1.602724e+09
8,8,vaini,-34.42,19.23,55.99,92.0,11.0,1.99,ZA,1.602724e+09
9,9,hobart,-21.20,-175.20,77.00,69.0,75.0,8.05,TO,1.602724e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [93]:
coord = weather_data_df[["Lat", "Lng"]].astype(float)
weight = weather_data_df["Humidity"].astype(float)

In [106]:
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(coord, weights=weight, 
                               dissipating=True, max_intensity=20,
                              point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
hotel_df = weather_data_df.loc[(weather_data_df["Max Temp"]>70) & (weather_data_df["Max Temp"]<80) 
                                    & (weather_data_df["Wind Speed"]<10) & (weather_data_df["Cloudiness"]==0)]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
89,89,upernavik,31.93,70.46,71.73,26.0,0.0,1.95,PK,1.602724e+09
100,100,kaniama,25.36,55.40,77.00,37.0,0.0,2.24,AE,1.602724e+09
206,206,bud,28.81,70.52,71.35,23.0,0.0,5.21,PK,1.602724e+09
267,267,pascagoula,27.67,80.47,75.13,69.0,0.0,2.44,IN,1.602724e+09
341,341,geraldton,29.03,21.55,71.83,37.0,0.0,8.48,LY,1.602724e+09
345,345,tyup,24.47,54.37,78.80,34.0,0.0,3.36,AE,1.602724e+09
363,363,lumphat,29.54,67.88,73.36,18.0,0.0,5.44,PK,1.602724e+09
392,392,cache creek,-10.13,-36.18,74.89,80.0,0.0,9.31,BR,1.602724e+09
403,403,mergui,29.75,78.53,71.26,65.0,0.0,4.72,IN,1.602724e+09
513,513,linhai,-14.88,47.99,73.33,77.0,0.0,4.14,MG,1.602724e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

<ipython-input-50-64598bec773a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
89,89,upernavik,31.93,70.46,71.73,26.0,0.0,1.95,PK,1.602724e+09,
100,100,kaniama,25.36,55.40,77.00,37.0,0.0,2.24,AE,1.602724e+09,
206,206,bud,28.81,70.52,71.35,23.0,0.0,5.21,PK,1.602724e+09,
267,267,pascagoula,27.67,80.47,75.13,69.0,0.0,2.44,IN,1.602724e+09,
341,341,geraldton,29.03,21.55,71.83,37.0,0.0,8.48,LY,1.602724e+09,


In [57]:
def nearby(params):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    data = requests.get(base_url, params=params).json()
    return data

params = {
    'key' : g_key,
    'radius' : 5000,
    'type' : 'Hotel'}

for index, row in hotel_df.iterrows():
    Lat= row["Lat"]
    Lng= row["Lng"]
    params['location']=f"{Lat},{Lng}"
    data=nearby(params)
    
    try:
        hotel_df.loc[index, "Hotel Name"] = data['results'][0]['name']
    except:
        print(f"No hotel near this {City}")
        pass
    
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
89,89,upernavik,31.93,70.46,71.73,26.0,0.0,1.95,PK,1.602724e+09,Kulachi
100,100,kaniama,25.36,55.40,77.00,37.0,0.0,2.24,AE,1.602724e+09,Sharjah
206,206,bud,28.81,70.52,71.35,23.0,0.0,5.21,PK,1.602724e+09,Zāhir Pīr
267,267,pascagoula,27.67,80.47,75.13,69.0,0.0,2.44,IN,1.602724e+09,Maholi
341,341,geraldton,29.03,21.55,71.83,37.0,0.0,8.48,LY,1.602724e+09,Jalu
345,345,tyup,24.47,54.37,78.80,34.0,0.0,3.36,AE,1.602724e+09,Abu Dhabi
363,363,lumphat,29.54,67.88,73.36,18.0,0.0,5.44,PK,1.602724e+09,Sibi
392,392,cache creek,-10.13,-36.18,74.89,80.0,0.0,9.31,BR,1.602724e+09,Coruripe
403,403,mergui,29.75,78.53,71.26,65.0,0.0,4.72,IN,1.602724e+09,Kotdwar
513,513,linhai,-14.88,47.99,73.33,77.0,0.0,4.14,MG,1.602724e+09,Antsohihy


In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [107]:
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(coord, weights=weight, 
                               dissipating=True, max_intensity=20,
                              point_radius=3)
fig.add_layer(heat_layer)

# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))